# Import modules/libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

C:\Users\ajitp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load data into memory with Pandas

In [4]:
test = pd.read_csv('Store_Test_Forecast_Class.csv')
train = pd.read_csv('Store_Train_Forecast_Class.csv')

### Investigate dataframes

In [5]:
print (test.shape)
test.head()

(5681, 11)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [6]:
print (train.shape)
train.head()

(8523, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##### <font color="red"> Note: Train data contains the Item_Outlet_Sales </font>
##### <font color="orange"> It means its a regression problem </font>

### So, let's start with training data first, because it can be used to quantify the sales amount for different products

#### On the first look, I have discovered few categorical columns to group the sales upon
1. Outlet_Location_Type - done
2. Outlet_Size - done
3. Outlet_Identifier - done
4. Outlet_Type - done
5. Item_Type
6. Item_Fat_Content

#### 1. Lets check the sales table for null values

In [63]:
print ("No. of Null values in Item_Outlet_Sales:  ", train.Item_Outlet_Sales.isna().sum())
print ("Ratio of Null values in Item_Outlet_Sales:", train.Item_Outlet_Sales.isna().sum()/train.shape[0])

No. of Null values in Item_Outlet_Sales:   0
Ratio of Null values in Item_Outlet_Sales: 0.0


#### 2. Let's calculate the total sales

In [71]:
print ("Total sales volume (Item_Outlet_Sales): CAD", round(train.Item_Outlet_Sales.sum()/1000000, 2), "millions")

Total sales volume (Item_Outlet_Sales): CAD 18.59 millions


#### 3. Group by "Outlet_Location_Type" and divide the sum by 1000,000 to convert the figures to millions

In [115]:
train.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].sum().divide(1000000)

# Verification code: Sales volume 
#train.groupby(['Outlet_Location_Type'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Location_Type
Tier 1    4.482059
Tier 2    6.472314
Tier 3    7.636753
Name: Item_Outlet_Sales, dtype: float64

#### 4. Group by "Outlet_Size", and convert total sales to millions

In [85]:
train.groupby(['Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000)

# Verification code: Sales volume 
# train.groupby(['Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Size
High      2.142664
Medium    7.489719
Small     4.566212
Name: Item_Outlet_Sales, dtype: float64

#### 5. Group by "Outlet_Size"& "Outlet_Location_Type", and convert total sales to millions

In [117]:
train.groupby(['Outlet_Location_Type','Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000)
#train.groupby(['Outlet_Location_Type','Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000).sort_values()

# Verification code: Sales volume 
# train.groupby(['Outlet_Location_Type','Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Location_Type  Outlet_Size
Tier 1                Medium         2.183970
                      Small          2.298089
Tier 2                Small          2.268123
Tier 3                High           2.142664
                      Medium         5.305749
Name: Item_Outlet_Sales, dtype: float64

###### <font color='red'> <b> Findings: </b> Now it tells confusing story, Tier 1 has Medium/Small outlets, whereas Tier 3 has Medium/High outlets. <font>

#### 6. Group by "Outlet_Identifier", and convert total sales to millions

In [120]:
train.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].sum().divide(1000000).sort_values(ascending=False)

# Verification code: Sales volume 
#train.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].sum().divide(1000000).sort_values(ascending=False).sum()

Outlet_Identifier
OUT027    3.453926
OUT035    2.268123
OUT049    2.183970
OUT017    2.167465
OUT013    2.142664
OUT046    2.118395
OUT045    2.036725
OUT018    1.851823
OUT010    0.188340
OUT019    0.179694
Name: Item_Outlet_Sales, dtype: float64

###### <font color='red'> <b> Findings: </b> OUT027 has highest sales. Let's check its location. </font>

#### 7. Verifying the accuracy of data, if an Outliet has Multiple Tiers assigned or not

In [89]:
train.groupby(['Outlet_Identifier','Outlet_Location_Type'])['Item_Outlet_Sales'].count()

# Verification code: Sales volume 
#train.groupby(['Outlet_Identifier','Outlet_Location_Type'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Identifier  Outlet_Location_Type
OUT010             Tier 3                  555
OUT013             Tier 3                  932
OUT017             Tier 2                  926
OUT018             Tier 3                  928
OUT019             Tier 1                  528
OUT027             Tier 3                  935
OUT035             Tier 2                  930
OUT045             Tier 2                  929
OUT046             Tier 1                  930
OUT049             Tier 1                  930
Name: Item_Outlet_Sales, dtype: int64

###### <font color='red'> <b>  Finding:</b> It finds that all stores have only one Tier Type assigned.</font>

#### 8. Verifying the accuracy of data, if an Outliet has Multiple Outlet_Sizes assigned or not

In [91]:
train.groupby(['Outlet_Identifier','Outlet_Size'])['Item_Outlet_Sales'].count()

# Verification code: Sales volume 
#train.groupby(['Outlet_Identifier','Outlet_Size'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Identifier  Outlet_Size
OUT013             High           932
OUT018             Medium         928
OUT019             Small          528
OUT027             Medium         935
OUT035             Small          930
OUT046             Small          930
OUT049             Medium         930
Name: Item_Outlet_Sales, dtype: int64

###### <font color='red'> <b> Finding:</b> It finds that something is wrong with outlet_size column. Let's investigate that.</font>

#### 9. Check the count of NaN values in Outlet_Size

In [92]:
train.Outlet_Size.value_counts(dropna=False).divide(train.shape[0]).multiply(100)

Medium    32.770151
NaN       28.276428
Small     28.018303
High      10.935117
Name: Outlet_Size, dtype: float64

###### <font color='red'> <b> Findings:</b> So it turned out that Outlet_Size is non-significant/non-appropriate column to work with because it contains more than 28% missing values.</font>

#### 10. Group by "Item_Type" to analyze the share of each category in total sales

In [112]:
train.groupby(['Item_Type'])['Item_Outlet_Sales'].sum().divide(1000000).sort_values(ascending=False)

# Verification code: Sales volume 
#train.groupby(['Item_Type'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Item_Type
Fruits and Vegetables    2.820060
Snack Foods              2.732786
Household                2.055494
Frozen Foods             1.825735
Dairy                    1.522594
Canned                   1.444151
Baking Goods             1.265525
Health and Hygiene       1.045200
Meat                     0.917566
Soft Drinks              0.892898
Breads                   0.553237
Hard Drinks              0.457793
Starchy Foods            0.351401
Others                   0.325518
Breakfast                0.232299
Seafood                  0.148868
Name: Item_Outlet_Sales, dtype: float64

###### <font color='red'> Findings: Clear Insight into Item Type Sales. </font>

#### 11. Group by "Outlet_Identifier & Item_Type", and convert total sales to millions

##### Differentiate the sales volume by "Item_Type" in each store

In [108]:

train.groupby(['Outlet_Identifier', 'Item_Type'])['Item_Outlet_Sales'].sum().divide(1000000)

#Save Result to CSV
#train.groupby(['Outlet_Identifier', 'Item_Type'])['Item_Outlet_Sales'].sum().divide(1000000).to_csv('verify_sales_on_item_type_and_outlet.csv')

# Verification code: Sales volume 
#train.groupby(['Outlet_Identifier', 'Item_Type'])['Item_Outlet_Sales'].sum().divide(1000000).sum()

Outlet_Identifier  Item_Type    
OUT010             Baking Goods     0.010693
                   Breads           0.007657
                   Breakfast        0.004081
                   Canned           0.009020
                   Dairy            0.015307
                                      ...   
OUT049             Others           0.047772
                   Seafood          0.011827
                   Snack Foods      0.350244
                   Soft Drinks      0.097586
                   Starchy Foods    0.042020
Name: Item_Outlet_Sales, Length: 160, dtype: float64

###### <font color='red'> <b> Findings: This data is good and very clear to understand. But its may require a inline change to view all the entries in Jupyter Notebook. Alternatively, I'm saving it to csv to view the results in excel. </font>

In [ ]:
### Combine Test and Train dataframes@